In [1]:
import os

import numpy as np
import pandas as pd


import re
import gc

import tensorflow as tf
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing import sequence


2023-04-14 04:53:44.131061: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 04:53:45.206283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-04-14 04:53:45.206402: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)   

4 Physical GPUs, 4 Logical GPUs


2023-04-14 04:53:50.940171: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:53:50.940630: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:53:50.940965: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:53:50.941313: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:53:50.952801: I tensorflow/compiler/xla/stream_executo

tor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:53:53.344654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:53:53.345001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:53:53.345353: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:53:53.345682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from Sy

In [3]:
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:05.0, compute capability: 7.0
/job:localhost/replica:0/task:0/device:GPU:2 -> device: 2, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:06.0, compute capability: 7.0
/job:localhost/replica:0/task:0/device:GPU:3 -> device: 3, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:07.0, compute capability: 7.0



2023-04-14 04:54:13.136907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:54:13.137405: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:54:13.137776: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:54:13.138188: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-14 04:54:13.138563: I tensorflow/compiler/xla/stream_executo

In [1]:
# tf.config.list_physical_devices('GPU')

In [2]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [4]:
train = pd.read_csv('./data/train.csv', header=None, names=['text','time','label'])
test = pd.read_csv('./data/test.csv', header=None, names=['text','time','label'])

In [5]:
X_train_list = train['text'].tolist()
X_test_list = test['text'].tolist()
y_train = train['label'].tolist()
y_test = test['label'].tolist()

In [ ]:
del train 
del test

In [3]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

In [7]:
M = 'allenai/longformer-base-4096'

In [8]:
tokenizer = AutoTokenizer.from_pretrained(M)

In [9]:
X_train = tokenizer(X_train_list, truncation=True, padding=True)
# X_test = tokenizer(X_test, truncation=True, padding=True)

In [11]:
import pickle

In [ ]:
with open('X_train.pkl','rb') as f:
    X_train = pickle.load(f)

In [ ]:
del X_train_list
del X_test_list

In [ ]:
n = len(X_train['input_ids'])

In [18]:
trainset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),
    y_train
))

# val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(X_test),
#     y_test
# ))

In [14]:
id2label = {0: "NG", 1: "Groomer"}
label2id = {"NG": 0, "Groomer": 1}

In [12]:
from tensorflow.keras.optimizers.legacy import Adam


In [13]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(M, num_labels=2, id2label=id2label, label2id=label2id)
    optimizer = Adam(learning_rate=0.0001) #learning_rate=5e-5
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


Some layers from the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerForSequenceClassification: ['lm_head']
- This IS expected if you are initializing TFLongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFLongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [14]:
model.fit(
    trainset.shuffle(n).batch(1), epochs=1, batch_size=1,
    # validation_data = val_dataset.shuffle(len(X_test['input_ids'])).batch(32),
    # callbacks = [callback_earlystop]
)

2023-04-14 01:59:59.002910: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_3"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
input: "Placeholder/_2"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
      type: DT_INT32
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 10
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 610
        }
      }
      shape {
        dim {
          size: 610
        }
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  ty

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [18]:
# model.save('lf_1')

OperatorNotAllowedInGraphError: Exception encountered when calling layer 'longformer' (type TFLongformerMainLayer).

Using a symbolic `tf.Tensor` as a Python `bool` is not allowed: AutoGraph did convert this function. This might indicate you are trying to use an unsupported feature.

Call arguments received by layer 'longformer' (type TFLongformerMainLayer):
  • args=('tf.Tensor(shape=(None, None), dtype=int64)',)
  • kwargs={'attention_mask': 'tf.Tensor(shape=(None, None), dtype=int64)', 'head_mask': 'None', 'global_attention_mask': 'tf.Tensor(shape=(None, None), dtype=int64)', 'token_type_ids': 'None', 'position_ids': 'None', 'inputs_embeds': 'None', 'output_attentions': 'False', 'output_hidden_states': 'False', 'return_dict': 'True', 'training': 'False'}

In [ ]:
# model.save_weights('savem/lf_w2')

In [15]:
new_model = TFAutoModelForSequenceClassification.from_pretrained(M, num_labels=2, id2label=id2label, label2id=label2id)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) #learning_rate=5e-5
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
new_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Some layers from the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerForSequenceClassification: ['lm_head']
- This IS expected if you are initializing TFLongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFLongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
new_model.load_weights('savem/lf_w1')


In [ ]:
from transformers import TextClassificationPipeline

# 로드하기

text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer, 
    model=model, 
    framework='tf',
    return_all_scores=True
)

In [ ]:
text_classifier('hello')

In [15]:
gc.collect()

21

In [55]:
import inspect

In [58]:
print(inspect.getsource(new_model.__call__))  

    @traceback_utils.filter_traceback
    def __call__(self, *args, **kwargs):
        if self._layout_map is not None and not self.built:
            # Note that this method is only overridden for DTensor and layout
            # injection purpose.
            # Capture the inputs and create graph input as replacement for model
            # to initialize its weights first.
            copied_args = copy.copy(args)
            copied_kwargs = copy.copy(kwargs)

            (
                inputs,
                copied_args,
                copied_kwargs,
            ) = self._call_spec.split_out_first_arg(copied_args, copied_kwargs)

            def _convert_to_graph_inputs(x):
                if isinstance(x, (tf.Tensor, np.ndarray, float, int)):
                    x = tf.convert_to_tensor(x)
                    return input_layer_module.Input(x.shape)

            # TODO(scottzhu): maybe better handle mask and training flag.
            inputs = tf.nest.map_structure(_convert_to

In [57]:
print(inspect.getsource(new_model.fit))  

    @traceback_utils.filter_traceback
    def fit(
        self,
        x=None,
        y=None,
        batch_size=None,
        epochs=1,
        verbose="auto",
        callbacks=None,
        validation_split=0.0,
        validation_data=None,
        shuffle=True,
        class_weight=None,
        sample_weight=None,
        initial_epoch=0,
        steps_per_epoch=None,
        validation_steps=None,
        validation_batch_size=None,
        validation_freq=1,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
    ):
        """Trains the model for a fixed number of epochs (iterations on a dataset).

        Args:
            x: Input data. It could be:
              - A Numpy array (or array-like), or a list of arrays
                (in case the model has multiple inputs).
              - A TensorFlow tensor, or a list of tensors
                (in case the model has multiple inputs).
              - A dict mapping input names to the correspond

In [19]:
a = tf.convert_to_tensor(X_train['input_ids'])
del a

2023-04-14 05:02:55.748295: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1096531968 exceeds 10% of free system memory.


In [33]:
y_pred = new_model.predict(a[:1000], batch_size=10)


100/100 [==============================] - 217s 2s/step


In [34]:
y_pred_score = [np.argmax(e) for e in y_pred[0]]

In [ ]:
# test_loss, test_acc = new_model.evaluate(X_train['input_ids'],y_train verbose=2)

In [35]:
y_pred_score

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [36]:
from sklearn.metrics import f1_score, classification_report


f1 = f1_score(y_train[:1000],y_pred_score)
f1

0.0

In [42]:
y_pred

TFLongformerSequenceClassifierOutput(loss=None, logits=array([[ 1.9855249, -1.9520394],
       [ 1.985525 , -1.9520394],
       [ 1.9855248, -1.9520394],
       ...,
       [ 1.9855248, -1.9520395],
       [ 1.9855248, -1.9520394],
       [ 1.9855248, -1.9520394]], dtype=float32), hidden_states=None, attentions=None, global_attentions=None)

In [40]:
print(classification_report(y_train[:1000], y_pred_score, target_names=['NG','G']))

              precision    recall  f1-score   support

          NG       0.97      1.00      0.98       970
           G       0.00      0.00      0.00        30

    accuracy                           0.97      1000
   macro avg       0.48      0.50      0.49      1000
weighted avg       0.94      0.97      0.96      1000



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
